In [ ]:
import pandas as pd

def import_csv(url):
  path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
  return pd.read_csv(path)

orderlines_url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
orders_url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
products_url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
brands_url = "https://drive.google.com/file/d/1XGyabaa4mAkjixMk3XPgx_14OoSse3rs/view?usp=sharing"

orderlines = import_csv(orderlines_url)
orders = import_csv(orders_url)
products = import_csv(products_url)
brands = import_csv(brands_url)

In [ ]:
orderlines_df = orderlines.copy()

In [ ]:
orders_df = orders.copy()

In [ ]:
products_df = products.copy()

In [ ]:
brands_df = brands.copy()

# 1.&nbsp; **Duplicates**
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [ ]:
# orders
orders_df.duplicated().sum()

np.int64(0)

In [ ]:
# orderlines
orderlines_df.duplicated().sum()

np.int64(0)

In [ ]:
# products
products_df.duplicated().sum()

np.int64(8746)

In [ ]:
# brands
brands_df.duplicated().sum()

np.int64(0)

## Result: products have duplicates




In [ ]:
products_df = products_df.drop_duplicates()

In [ ]:
# products check again
products_df.duplicated().sum()

np.int64(0)

# 2.&nbsp; **`.info()`**

## ORDERS:



In [ ]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


### **1.created_date is not datetime, 2.total_paid column had missing rows**

In [ ]:
orders_df['created_date'] = pd.to_datetime(orders_df['created_date'])
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226909 non-null  int64         
 1   created_date  226909 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.9+ MB


In [ ]:
# how much % of the date frame has missing rows
print(f"5 missing values represents {((orders_df.total_paid.isna().sum() / orders_df.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


### As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [ ]:
orders_df = orders_df.loc[~orders.total_paid.isna(), :] #remove the rows with no values

In [ ]:
orders_df.info() #check again

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


In [ ]:
orders_cl = orders_df.copy()

## ORDERLINES:

In [ ]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


### **unit_price & date formats are object. 1. unit_price must be float, 2. date must have datetime format.**

In [ ]:
orderlines_df['date'] = pd.to_datetime(orderlines_df['date'])


### ** Alternative: Solution with keeping the rows with 2 dots (not losing data)**

In [ ]:
# Function to detect multiple dots
import re

def has_multiple_dots(text):
    return bool(re.search(r'.*\..*\..*', str(text)))

# Function to FIX entries with multiple dots
def fix_multiple_dots(value):
    if has_multiple_dots(value):
        parts = str(value).split('.')
        return parts[0] + '.' + ''.join(parts[1:])  # Keep only the first dot
    return value

# Apply the fix
orderlines_df['unit_price'] = orderlines_df['unit_price'].apply(fix_multiple_dots)


# Convert to numeric and fill NaNs with 0
orderlines_df['unit_price'] = pd.to_numeric(orderlines_df['unit_price'], errors='coerce').fillna(0)


In [ ]:
orderlines_df.info() #check again

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


In [ ]:
orderlines_df.shape[0]

293983

### **REAL SOLUTION: Solution with removing the rows with 2 dots (losing data)**

In [ ]:
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

,count
unit_price,
False,257814
True,36169


In [ ]:
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)


<ipython-input-19-c0e2fb117633>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)


In [ ]:
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

The 2 dot problem represents 12.3% of the rows in our DataFrame


In [ ]:
# remove them
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]

orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [ ]:
orderlines_df.shape[0] # this much rows left

216250

In [ ]:
# now convert to numeric
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [ ]:
orderlines_df.info() #check again

<class 'pandas.core.frame.DataFrame'>
Index: 216250 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  int64         
 1   id_order          216250 non-null  int64         
 2   product_id        216250 non-null  int64         
 3   product_quantity  216250 non-null  int64         
 4   sku               216250 non-null  object        
 5   unit_price        216250 non-null  float64       
 6   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 13.2+ MB


In [ ]:
orderlines_cl = orderlines_df.copy()

## PRODUCTS:

In [ ]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 non-null  object
dtypes: int64(1), object(6)
memory usage: 661.2+ KB


### **1. price & promo_price are objects. They must be numeric. 2. desc, price and type have missing rows.**

In [ ]:
# missing values
products_df["desc"].isna().sum()

np.int64(7)

In [ ]:
# 7 is a very small number to have missing, let's have a closer look
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107,814.659,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1.568.206,1.568.206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.35,5.659.896,0,12175397
18152,KAN0034-A,Open - Kanex USB-C Gigabit Ethernet Adapter Ma...,NaN,29.99,237.925,0,1298
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.99,22.99,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.99,56.99,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.99,1.441.174,0,11905404


### **We have 2 choices here:**

We can quickly and easily remove these rows.
Or, alternatively, the products names here are quite descriptive, so I'm tempted to just copy them to the description column, so that there is a description if we later want utilise this column. I wouldn't recommend this if this DataFrame was the source of truth for our website. But this is not the case here, and we're not faking any information (guessing a price or so), so I'm happy with this option

In [ ]:
products_df.loc[products_df['desc'].isna(), 'desc'] = products_df.loc[products_df['desc'].isna(), 'name']

In [ ]:
products_df.loc[products_df['desc'].isna(), :]

,sku,name,desc,price,promo_price,in_stock,type


In [ ]:
products_df["desc"].isna().sum()

np.int64(0)

Did you also notice above that we have the dreaded two decimal point problem in both the price and promo_price columns? We can also see prices with 3 decimal places, prices should have 2 decimal places: this gives us more cause for concern

In [ ]:
products_df.price.isna().sum()

np.int64(46)

In [ ]:
missing_percent = products_df.price.isna().value_counts(normalize=True).loc[True] * 100
print(f"The missing values in price are {missing_percent.round(2)}% of all rows in the DataFrame")


The missing values in price are 0.43% of all rows in the DataFrame


### Let's simply delete these rows to ensure that we can trust the numbers in our final DataFrame. Afterall, the price is very important when investigating discounts.

In [ ]:
products_df = products_df.loc[~products['price'].isna()]

Price: First, let's see how many values are affected by the 2-decimal-dot problems or 3 decimal places.

In [ ]:
price_problems_number = products_df.loc[(products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
price_problems_number

542

In [ ]:
print(f"The column price has in total {price_problems_number} wrong values. This is {round(((price_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column price has in total 542 wrong values. This is 5.15% of the rows of the DataFrame


### 5.15% is a reasonable amount of our data. However, the price column will be important to understanding discounts, so I'd like it to be very trustworthy as we are basing business decisions on it. Therefore, we'll delete these rows

In [ ]:
products_df = products_df.loc[(~products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :]

In [ ]:
products_df["price"] = pd.to_numeric(products_df["price"])

### promo_price: Again, let's begin by seeing how many values are affected by the 2-decimal-dots problem, or the 3 decimal-places problem

In [ ]:
promo_problems_number = products_df.loc[(products_df.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products_df.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
promo_problems_number

9232

In [ ]:
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products_df.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column promo_price has in total 9232 wrong values. This is 92.39% of the rows of the DataFrame


### better to delete the column then

In [ ]:
products_cl = products_df.drop(columns=["promo_price"])

In [ ]:
products_cl.info()  #check again

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       9992 non-null   object 
 1   name      9992 non-null   object 
 2   desc      9992 non-null   object 
 3   price     9992 non-null   float64
 4   in_stock  9992 non-null   int64  
 5   type      9946 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 546.4+ KB


In [ ]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [ ]:
brands_cl = brands_df.copy()

# 3.&nbsp; Don't forget to download/save your new DataFrames. Also, give them an obvious name, so that you know they are the cleaned version and not the original DataFrame.

In [ ]:
#from google.colab import files

#orders_df.to_csv("orders_cl.csv", index=False)
#files.download("orders_cl.csv")

#orderlines_df.to_csv("orderlines_cl.csv", index=False)
#files.download("orderlines_cl.csv")

#products_cl.to_csv("products_cl.csv", index=False)
#files.download("products_cl.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>